In [1]:
# pip install pyspark

  Using cached pyspark-3.5.1.tar.gz (317.0 MB)
  Preparing metadata (setup.py) ... done
  Using cached py4j-0.10.9.7-py2.py3-none-any.whl (200 kB)
Using legacy 'setup.py install' for pyspark, since package 'wheel' is not installed.
  Running setup.py install for pyspark ... done
--- Logging error ---
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/pip/_internal/utils/logging.py", line 177, in emit
    self.console.print(renderable, overflow="ignore", crop=False, style=style)
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/pip/_vendor/rich/console.py", line 1673, in print
    extend(render(renderable, render_options))
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/pip/_vendor/rich/console.py", line 1305, in render
    for render_output in iter_render:
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-pack

In [3]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import window, sum

In [5]:
# create the spark session
spark = SparkSession.builder \
    .appName("Time_Based_Aggregations") \
    .getOrCreate()

# # read in csv file
# ecommerce_df = spark.read \
#     .format("csv") \
#     .option("header", "true") \
#     .load('/Users/jacquelineyu/src/kaizenflow/sorrentum_sandbox/projects/SorrTask792_Windowed_Streaming_Analysis_With_Apache_Spark_Streaming/archive/Amazon Sale Report.csv')

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/pyspark/bin/spark-class: line 71: /Library/Java/JavaVirtualMachines//bin/java: No such file or directory
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/pyspark/bin/spark-class: line 97: CMD: bad array subscript
head: illegal line count -- -1


PySparkRuntimeError: [JAVA_GATEWAY_EXITED] Java gateway process exited before sending its port number.

In [11]:
# list columns to drop
col_drop = ['Order ID', 'Style', 'SKU', 'ASIN', 'Courier Status', 'promotion-ids', 'B2B', 'fulfilled-by', 'Unnamed: 22']

# drop the columns
ecommerce_df = ecommerce_df.drop(*col_drop)

In [13]:
ecommerce_df.show(5)

+-----+--------+--------------------+----------+--------------+------------------+-------------+----+---+--------+------+-----------+-----------+----------------+------------+
|index|    Date|              Status|Fulfilment|Sales Channel |ship-service-level|     Category|Size|Qty|currency|Amount|  ship-city| ship-state|ship-postal-code|ship-country|
+-----+--------+--------------------+----------+--------------+------------------+-------------+----+---+--------+------+-----------+-----------+----------------+------------+
|    0|04-30-22|           Cancelled|  Merchant|     Amazon.in|          Standard|          Set|   S|  0|     INR|647.62|     MUMBAI|MAHARASHTRA|        400081.0|          IN|
|    1|04-30-22|Shipped - Deliver...|  Merchant|     Amazon.in|          Standard|        kurta| 3XL|  1|     INR| 406.0|  BENGALURU|  KARNATAKA|        560085.0|          IN|
|    2|04-30-22|             Shipped|    Amazon|     Amazon.in|         Expedited|        kurta|  XL|  1|     INR| 329.0

In [14]:
# Convert 'Date' column to timestamp type
ecommerce_df = ecommerce_df.withColumn("Date", ecommerce_df["Date"].cast("timestamp"))

# time-based aggregations using tumbling windows
# aggregate the sales using 'Amount' and 'Qty' for each day
tumbling_window_df = ecommerce_df \
    .groupBy(window("Date", "1 day")) \
    .agg(sum("Amount").alias("TotalAmount"), sum("Qty").alias("TotalQty"))

# time-based aggregations using sliding windows
# aggregate the sales using 'Amount' and 'Qty' for each day using sliding window of 3 days
sliding_window_df = ecommerce_df \
    .groupBy(window("Date", "3 days", slideDuration="1 day")) \
    .agg(sum("Amount").alias("TotalAmount"), sum("Qty").alias("TotalQty"))

# print tumbling window df
print("Tumbling Window Aggregations:")
tumbling_window_df.show(truncate=False)

# print sliding window df
print("Sliding Window Aggregations:")
sliding_window_df.show(truncate=False)

Tumbling Window Aggregations:


+------+-----------+--------+
|window|TotalAmount|TotalQty|
+------+-----------+--------+
+------+-----------+--------+

Sliding Window Aggregations:


+------+-----------+--------+
|window|TotalAmount|TotalQty|
+------+-----------+--------+
+------+-----------+--------+



In [ ]:
# stop the SparkSession when the app is done
spark.stop()